# Superdense coding walkthrough

This notebook demonstrates the full superdense coding protocol using Qiskit. We build a Bell pair (Hadamard on qubit 0, followed by CNOT control→target 1), encode two classical bits on Alice's qubit with Pauli operations, and let Bob decode with a Bell-basis measurement.


In [ ]:
# Setup
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

backend = AerSimulator()

# Bit→gate map for Alice
BIT_TO_OP = {
    '00': [],      # I
    '01': ['x'],   # X
    '10': ['z'],   # Z
    '11': ['x', 'z'],  # XZ (equivalent to Y up to phase)
}


## Build the protocol circuit

1. **Entanglement:** Apply H on qubit 0 then CNOT(0→1) to create a Bell pair.
2. **Encoding (Alice):** Map the two classical bits to I, X, Z, or XZ on her qubit.
3. **Decoding (Bob):** Apply CNOT(0→1) then H on Alice's qubit, then measure both qubits to recover the two bits.


In [ ]:
def build_superdense(msg: str) -> QuantumCircuit:
    """Return a superdense coding circuit for a given 2-bit message."""
    if msg not in BIT_TO_OP:
        raise ValueError("msg" must be one of "00", "01", "10", "11")
    qc = QuantumCircuit(2, 2, name=f"SDC_{msg}")  # q0 = Alice, q1 = Bob

    # 1) Share Bell pair (entanglement resource)
    qc.h(0)
    qc.cx(0, 1)

    # 2) Alice encodes her classical bits on her qubit
    if "x" in BIT_TO_OP[msg]:
        qc.x(0)
    if "z" in BIT_TO_OP[msg]:
        qc.z(0)

    qc.barrier(label="Alice sends q0 → Bob")

    # 3) Bob decodes (Bell-basis measurement)
    qc.cx(0, 1)
    qc.h(0)
    qc.measure([0, 1], [0, 1])  # c0 <- q0, c1 <- q1

    return qc

# Draw an example circuit
example = build_superdense("10")
print(example.draw())


## Validate all four messages

The decoder should recover the exact two-bit payload. The assertion checks that the most probable result equals the input message (tolerating simulator shot noise otherwise).


In [ ]:
def run_message(msg: str, shots: int = 2048):
    qc = build_superdense(msg)
    tqc = transpile(qc, backend)
    result = backend.run(tqc, shots=shots).result()
    counts = result.get_counts()

    # Identify the most likely outcome and ensure it matches the payload
    top_bitstring = max(counts, key=counts.get)
    assert top_bitstring == msg, f"Decoded {top_bitstring} instead of {msg}"
    success_rate = counts.get(msg, 0) / shots
    return counts, success_rate

summary = {}
for message in BIT_TO_OP:
    counts, success = run_message(message)
    summary[message] = {"counts": counts, "success": success}
    print(f"Message {message} → success ~ {success:.3f}, counts {counts}")

# Visualize histograms per message
plots = {m: plot_histogram(data["counts"], title=f"Decoded distribution for {m}") for m, data in summary.items()}
plots


### Why it works

- The shared Bell pair creates correlations that let two classical bits ride on one qubit.
- Alice's Pauli operations rotate the entangled state into one of four orthogonal Bell states, uniquely identifying her message.
- Bob's inverse Bell-basis measurement (CNOT + Hadamard) maps those Bell states back to computational basis states so measuring both qubits reveals the original two bits.
